# Generate Splits

In [ ]:
import os
import pandas as pd

In [ ]:
username = ""
source = ""
media = ""

In [ ]:
data_path = os.path.join("../../data/recommendations", username)
def fullpath(x):
    return os.path.join(data_path, f"{x}.csv")

In [ ]:
df = pd.read_csv(fullpath(f"user_{media}_list"))
df = df.sort_values(by='timestamp')

In [ ]:
df.loc[lambda x: x.status == 1].to_csv(fullpath(f"{media}.ptw"), index=False)

In [ ]:
df.loc[lambda x: (x.status != 1) & (x['score'] == 0)].to_csv(fullpath(f"{media}.implicit"), index=False)

In [ ]:
df.loc[lambda x: (x.status != 1) & (x['score'] != 0)].to_csv(fullpath(f"{media}.explicit"), index=False)